# Data cleaning Immobiliare.it

I first import the libraries that i believe they will be needed for this cleaning process.

In [203]:
import numpy as np
import pandas as pd
import re
from datetime import date

I now need to load the csv file into a pandas dataframe and then check what the dataframe looks like as well as its datatypes.

In [204]:
#REMEMBER TO FILL THE PROPER FILE NAME!
df_raw = pd.read_csv('house_prices_italy_2023_7.csv')
df_raw.head()

,Unnamed: 0,region,city,area,rooms,toilets,price,date
0,0,abruzzo,Pescara,295m²,5+,3+,€ 257.000,2023-07-31
1,1,abruzzo,Francavilla al Mare,88m²,3,1,€ 168.000,2023-07-31
2,2,abruzzo,Pescara,227m²,5,3+,€ 299.000,2023-07-31
3,3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43m²,2 - 4,1,da € 165.000,2023-07-31
4,4,abruzzo,Francavilla al Mare,83m²,4,1,€ 139.000,2023-07-31


In [205]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  39999 non-null  int64 
 1   region      39999 non-null  object
 2   city        39999 non-null  object
 3   area        39974 non-null  object
 4   rooms       39057 non-null  object
 5   toilets     39999 non-null  object
 6   price       39999 non-null  object
 7   date        39999 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.4+ MB


As expected some columns have wrong datatypes for what I plan to do. I would like to have them numerical columns into integer format with np.nan values for null. Date column as date instead.
Let's first have a look on how many nulls we have here:

In [206]:
df_raw.isnull().sum()

Unnamed: 0      0
region          0
city            0
area           25
rooms         942
toilets         0
price           0
date            0
dtype: int64

Apart from rooms column, not so many indeed. I decided to drop the first column (*Unnamed:0*) since it does not carry any info:

In [207]:
df=df_raw.drop("Unnamed: 0", axis=1)

I change the column names to match the dimensions of the values:

In [208]:
df.rename(columns={'area':'area[m2]', 'price':'price[€]'}, inplace=True)

Now I can start the cleaning in column order. I need to end up with a simple integer number.
I force the "area" column as 'string' datatype, if not, later I noticed it will raise errors.

In [209]:
df['area[m2]'] = df['area[m2]'].astype(str)

In [210]:
#compilo il pattern
p = re.compile('[0-9]+')


df['area[m2]'] = df['area[m2]'].apply(lambda x: 0 if p.search(x) is None else p.search(x).group())
df.head()

,region,city,area[m2],rooms,toilets,price[€],date
0,abruzzo,Pescara,295,5+,3+,€ 257.000,2023-07-31
1,abruzzo,Francavilla al Mare,88,3,1,€ 168.000,2023-07-31
2,abruzzo,Pescara,227,5,3+,€ 299.000,2023-07-31
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,da € 165.000,2023-07-31
4,abruzzo,Francavilla al Mare,83,4,1,€ 139.000,2023-07-31


In [211]:
df['area[m2]'] = df['area[m2]'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39999 non-null  object
 1   city      39999 non-null  object
 2   area[m2]  39999 non-null  int64 
 3   rooms     39057 non-null  object
 4   toilets   39999 non-null  object
 5   price[€]  39999 non-null  object
 6   date      39999 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


Since I am not particularly interested in getting to know statistics about rooms and toilets but more on prices, area and their distribution over the italian territory, I will keep **rooms** and **toilets** columns as they are, namely categorical variables of type strings.
Now for for the **price** column: 


In [212]:
df['price[€]'] = df['price[€]'].str.replace('.','', regex=False)
df['price[€]'] = df['price[€]'].str.replace('€','', regex= False)

p_price = re.compile('\d+')
df['price[€]'] = df['price[€]'].apply(lambda x: 0 if x.strip().isnumeric() == False else p.search(x).group())
df['price[€]'] = df['price[€]'].astype('int')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39999 non-null  object
 1   city      39999 non-null  object
 2   area[m2]  39999 non-null  int64 
 3   rooms     39057 non-null  object
 4   toilets   39999 non-null  object
 5   price[€]  39999 non-null  int64 
 6   date      39999 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.1+ MB


In order to convert to integer I was forced to put *0* instead of ```np.nan```. That zero value will interfere with the statistic computation. Hence i convert it back to NaN.

In [213]:
#CONVERT 0 VALUES TO NAN
df['area[m2]'] = df['area[m2]'].apply(lambda x: np.nan if x==0 else x)
df['price[€]'] = df['price[€]'].apply(lambda x: np.nan if x==0 else x)

#CONVERT TO INT TYPE
df['area[m2]'] = df['area[m2]'].astype('Int64', errors='ignore')
df['price[€]'] = df['price[€]'].astype('Int64', errors='ignore')

df

,region,city,area[m2],rooms,toilets,price[€],date
0,abruzzo,Pescara,295,5+,3+,257000,2023-07-31
1,abruzzo,Francavilla al Mare,88,3,1,168000,2023-07-31
2,abruzzo,Pescara,227,5,3+,299000,2023-07-31
3,abruzzo,Appartamenti di nuova costruzione a Tortoreto,43,2 - 4,1,<NA>,2023-07-31
4,abruzzo,Francavilla al Mare,83,4,1,139000,2023-07-31
...,...,...,...,...,...,...,...
39994,veneto,San Zeno di Montagna,400,5+,3+,598000,2023-07-31
39995,veneto,Bassano del Grappa,260,5+,3+,440000,2023-07-31
39996,veneto,Garda,219,5+,2,1200000,2023-07-31
39997,veneto,Mirano,185,5+,2,450000,2023-07-31


In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    39999 non-null  object
 1   city      39999 non-null  object
 2   area[m2]  39974 non-null  Int64 
 3   rooms     39057 non-null  object
 4   toilets   39999 non-null  object
 5   price[€]  35339 non-null  Int64 
 6   date      39999 non-null  object
dtypes: Int64(2), object(5)
memory usage: 2.2+ MB


I now rename the regions into a more appropriate manner:

In [215]:
regions_dict = {'abruzzo':'Abruzzo', 'basilicata':'Basilicata', 'campania':'Campania', 'calabria':'Calabria', 'emilia-romagna':'Emilia Romagna',
       'friuli-venezia-giulia':'Friuli-Venezia Giulia', 'lazio': 'Lazio', 'liguria':'Liguria', 'lombardia':'Lombardia', 'marche':'Marche',
       'molise':'Molise', 'piemonte':'Piemonte', 'puglia':'Puglia', 'sardegna':'Sardegna', 'sicilia':'Sicilia', 'toscana':'Toscana',
       'trentino-alto-adige': 'Trentino-Alto Adige', 'umbria':'Umbria', 'valle-d-aosta':'Valle d\'Aosta', 'veneto':'Veneto'}

df.replace({'region':regions_dict}, inplace=True)
df.isna().sum()


region         0
city           0
area[m2]      25
rooms        942
toilets        0
price[€]    4660
date           0
dtype: int64

In [216]:
df.sample(20)

,region,city,area[m2],rooms,toilets,price[€],date
4254,Campania,Brusciano,90,4,2,295000,2023-07-31
3206,Basilicata,Matera,75,5+,3,90000,2023-07-31
18019,Marche,Falerone,345,5+,3,380000,2023-07-31
12187,Lazio,Roma,195,5,2,850000,2023-07-31
6772,Calabria,Lamezia Terme,144,5,2,139000,2023-07-31
1726,Abruzzo,Alba Adriatica,40,2,1,69000,2023-07-31
3914,Basilicata,Potenza,50,2,1,55000,2023-07-31
6842,Calabria,Crosia,78,3,1,39000,2023-07-31
31679,Toscana,Castiglione della Pescaia,653,5+,3+,1100000,2023-07-31
31206,Toscana,Bagno a Ripoli,159,5,2,700000,2023-07-31


I believe this is enough. I do not convert the **date** column into DateTime object since for Tableau it is perfectly capable of treating as such as it is. It will now be possible for me to work on a Dashboard.

I save the csv file:

In [141]:
year = date.today().year
month = date.today().month

In [142]:
df.to_csv(f'house_prices_italy_{year}_{month}_cleaned.csv')